In [1]:
from pathlib import Path
from llama_index import download_loader
import dotenv
import os
from llama_index import VectorStoreIndex

env_path = Path('./.env')

assert dotenv.load_dotenv(env_path),"No .env file found"

In [2]:
MarkdownReader = download_loader("MarkdownReader")

loader = MarkdownReader()


In [3]:
path = os.getenv("PATH_DOC")
documents = loader.load_data(file=Path(path))

In [4]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [5]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


In [6]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
#build index
index = VectorStoreIndex.from_documents(documents, show_progress=True)

d:\anaconda-envs\rag-implements\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 26/26 [00:04<00:00,  5.62it/s]


In [7]:
from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor



# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]

)



In [8]:
# query
queries = [
    "What is the meaning of life?",
    "Can I be arrested?"
]

response = query_engine.query("What is the meaning of life?")
print(response)
response.source_nodes



The meaning of life is a personal and subjective question that cannot be answered with the given context information.


[NodeWithScore(node=TextNode(id_='67d3e073-aada-431b-82d0-dad850567264', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='4523196b-b7db-45c1-8c2d-af101f639a73', node_type=None, metadata={}, hash='0eb2b77f2b4ed5c69eb067c955be83a42193e79ceb0e41d8603206c2926cf14a')}, hash='9a56dfc65a5b0e10a2f705fbc70c2eeb3837ed0ab20ad17a7bb47c5e6985acd9', text='All individuals/non-individuals (including foreign citizens/entities) earning taxable income in India\xa0must have a PAN card.', start_char_idx=0, end_char_idx=123, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.7312986539314421),
 NodeWithScore(node=TextNode(id_='30bd2d3b-fa0e-4476-87b1-33f81a4e62b5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='017

In [10]:
from llama_index.llms import OpenAI

llm = OpenAI(temperature=1, model="davinci")


In [11]:
?OpenAI

Init signature:
OpenAI(
    *,
    model: str = 'text-davinci-003',
    temperature: float = 0.0,
    max_tokens: Optional[int] = None,
    additional_kwargs: Dict[str, Any] = None,
    max_retries: int = 10,
) -> None
Docstring:      LLM interface.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           d:\anaconda-envs\rag-implements\lib\site-packages\llama_index\llms\openai.py
Type:           ModelMetaclass
Subclasses:     AzureOpenAI